In [1]:
import __init__

from collections import defaultdict
from functools import partial
from typing import Callable

import numpy as np
from cv2.typing import MatLike
from eval import cv2_to_pil, load_data, pil_to_cv2
from eval_with_ground import evaluate_image_pair_from_pil
from eval_without_ground import evaluate_image
from tqdm import tqdm

from common import DataSets
from dehaze import dehaze_and_enhance
from HE import he_enhance, ahe_enhance, clahe_enhance
from retinex import ssr_enhance, msr_enhance
from retinex_exp import ssr_enhance as ssr_exp_enhance
from retinex_exp import msr_enhance as msr_exp_enhance
from gan import gan_method

In [2]:
paired_samples = load_data()

100%|██████████████████████████████████████████████████████████████████████████████| 7200/7200 [04:24<00:00, 27.22it/s]


In [3]:
def eval(dataset: DataSets, method: Callable[[MatLike], MatLike]):
    samples = paired_samples[dataset]
    results = defaultdict(list)
    for sample in tqdm(samples):
        low_image = sample["low_image"]
        high_image = sample.get("high_image")
        # print(low_image, high_image)
        if low_image is not None:
            reference_img = cv2_to_pil(method(pil_to_cv2(low_image)))
            # reference_img.show()
            if high_image is not None:
                scores = evaluate_image_pair_from_pil(reference_img, high_image)
                # Compute metrics here
            else:
                scores = evaluate_image(reference_img, low_image)
            for key, s in scores.items():
                results[key].append(s)
    return results

In [4]:
paired_samples.keys()

dict_keys(['lol_dataset', 'Dark_Face', 'LOL-v2'])

### dahaze

In [6]:
result = eval('lol_dataset', dehaze_and_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:51<00:00,  4.47it/s]


In [7]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3874.5869 ± 3862.4163
PSNR: 14.2061 ± 4.2629
SSIM: 0.5058 ± 0.1518


In [8]:
result = eval('Dark_Face', dehaze_and_enhance)

100%|██████████████████████████████████████████████████████████████████████████████| 6000/6000 [43:29<00:00,  2.30it/s]


In [9]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 0.9235 ± 6.6517
CEI: 1.8998 ± 0.4300


In [10]:
result = eval('LOL-v2', dehaze_and_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.78it/s]


In [11]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2109.1912 ± 1787.1777
PSNR: 16.2343 ± 3.3398
SSIM: 0.5458 ± 0.1240


### HE

In [12]:
result = eval('lol_dataset', he_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.36it/s]


In [13]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2378.8735 ± 1416.1384
PSNR: 15.1016 ± 2.5712
SSIM: 0.4738 ± 0.1579


In [5]:
result = eval('Dark_Face', he_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:40<00:00, 12.40it/s]


In [6]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 86.3454 ± 0.7507
CEI: 3.9172 ± 3.0769


In [14]:
result = eval('LOL-v2', he_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.97it/s]


In [15]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3496.6123 ± 1771.7839
PSNR: 13.2393 ± 2.2080
SSIM: 0.4471 ± 0.1414


### AHE

In [ ]:
result = eval('lol_dataset', ahe_enhance)

  2%|█▍                                                                              | 9/500 [01:50<1:40:02, 12.23s/it]

In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

In [ ]:
result = eval('Dark_Face', ahe_enhance)

In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

In [ ]:
result = eval('LOL-v2', ahe_enhance)

In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

### CLAHE

In [19]:
result = eval('lol_dataset', clahe_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.43it/s]


In [20]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 7698.8047 ± 4759.8389
PSNR: 10.3307 ± 3.4654
SSIM: 0.4609 ± 0.1660


In [7]:
result = eval('Dark_Face', clahe_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:44<00:00, 11.35it/s]


In [8]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 39.7810 ± 22.3909
CEI: 1.4946 ± 0.2185


In [21]:
result = eval('LOL-v2', clahe_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.93it/s]


In [22]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3904.5256 ± 2796.8057
PSNR: 13.4324 ± 3.5129
SSIM: 0.5535 ± 0.1689


### SSR

In [23]:
result = eval('lol_dataset', ssr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [04:01<00:00,  2.07it/s]


In [24]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3915.6243 ± 2642.7864
PSNR: 13.1696 ± 2.9666
SSIM: 0.5459 ± 0.1487


In [9]:
result = eval('Dark_Face', ssr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [11:00<00:00,  1.32s/it]


In [10]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 80.9831 ± 1.9362
CEI: 2.5781 ± 1.8302


In [25]:
result = eval('LOL-v2', ssr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.17it/s]


In [26]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 5633.8906 ± 2985.6328
PSNR: 11.3565 ± 2.7842
SSIM: 0.5028 ± 0.1446


### MSR

In [27]:
result = eval('lol_dataset', msr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [12:55<00:00,  1.55s/it]


In [28]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 4078.6995 ± 2730.6064
PSNR: 12.9874 ± 2.9623
SSIM: 0.5251 ± 0.1488


In [11]:
result = eval('Dark_Face', msr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [41:28<00:00,  4.97s/it]


In [12]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 81.1224 ± 1.7313
CEI: 2.5951 ± 1.8311


In [29]:
result = eval('LOL-v2', msr_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:32<00:00,  1.53s/it]


In [30]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 5707.7998 ± 3010.6772
PSNR: 11.2794 ± 2.6881
SSIM: 0.4857 ± 0.1437


### SSR_exp

In [31]:
result = eval('lol_dataset', ssr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [03:52<00:00,  2.15it/s]


In [32]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2469.6917 ± 1579.3563
PSNR: 14.9746 ± 2.5514
SSIM: 0.5178 ± 0.1576


In [13]:
result = eval('Dark_Face', ssr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [11:30<00:00,  1.38s/it]


In [14]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 74.8235 ± 7.8909
CEI: 2.0988 ± 1.6271


In [33]:
result = eval('LOL-v2', ssr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:49<00:00,  2.01it/s]


In [34]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2080.4441 ± 1166.1401
PSNR: 15.5569 ± 2.2672
SSIM: 0.5381 ± 0.1454


### MSR_exp

In [35]:
result = eval('lol_dataset', msr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [12:53<00:00,  1.55s/it]


In [36]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2033.2766 ± 1403.8945
PSNR: 15.8799 ± 2.6242
SSIM: 0.5047 ± 0.1619


In [15]:
result = eval('Dark_Face', msr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [41:48<00:00,  5.01s/it]


In [16]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 76.3546 ± 6.2458
CEI: 2.1492 ± 1.6479


In [37]:
result = eval('LOL-v2', msr_exp_enhance)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:34<00:00,  1.54s/it]


In [38]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 1792.0731 ± 1005.6102
PSNR: 16.2654 ± 2.4641
SSIM: 0.5276 ± 0.1525
